In [1]:
%%time
import syft as sy
import torchaudio
import torch.optim as optim

import numpy as np
from glob import glob
from difflib import SequenceMatcher
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import librosa
import torch
import torch.nn as nn
from librosa.core import stft, magphase
from torch.autograd import Variable
from data_load import CodeSwitchDataset

import warnings
warnings.filterwarnings('ignore')

CPU times: user 2.15 s, sys: 217 ms, total: 2.37 s
Wall time: 3.91 s


/home/dexter/Desktop/Projects/CodeSwitching/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [7]:
from syft.frameworks.torch.fl.utils import federated_avg

In [3]:
class CodeSwitchDataset(Dataset):
    def __init__(self, lang, mode = "train", shuffle=True):
        self.mode = mode
        # data path
        self.lang = lang
        if self.lang == "Gujarati":
            self.max_len = 0
        elif self.lang == "Telugu":
            self.max_len = 617621
        elif self.lang == 'Tamil':
            self.max_len = 0
        else:
            raise Exception("Check Language")
        if self.mode == "train":
            self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        elif self.mode == "test":
            self.path = self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        else:
            raise Exception("Incorrect mode")
        self.file_list = os.listdir(os.path.join(self.path, 'Audio'))
        self.shuffle=shuffle
        self.csv_file = pd.read_csv(self.path + 'Transcription_LT_Sequence_Frame_Level_200_actual.tsv', header=None, sep='\t')
        self.input_length = []
        self.label_length = []
        

    def __len__(self):
        return len(self.csv_file)

    def pad(self, wav, trans, max_len):
        orig_len = len(wav)
        while len(wav) < max_len:
            diff = max_len - len(wav)
            ext = wav[:diff]
            wav = np.append(wav, wav[:diff])
            ratio = int(len(trans)*diff/len(wav))
            trans +=trans[:ratio]
        return wav, trans

    def preprocess(self, wav, sr, trans):

        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = [np.log(1 + x) for x in out]
        return np.array(out), trans

    def __getitem__(self, idx):
        file_name = self.csv_file[0][idx]
        trans = self.csv_file[1][idx]
        wav, sr = librosa.load(glob(self.path + 'Audio/*'+ str(file_name) + '.wav')[0])
        length_wav, length_trans = len(wav), len(trans)
        wav, trans = self.pad(wav, trans, self.max_len)
        if len(set(trans)) > 2:
            label = 1
        elif len(set(trans)) == 1 or len(set(trans)) == 2:
            label = 0
        else:
            raise Exception("Check transcript")
            
        if self.mode =="train":
            return (wav, sr, length_wav),  (trans, length_trans, len(trans))
        elif self.mode == "test":
            return wav
        else:
            raise Exception("Incorrect Mode")


class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        s 1
        e 2
        t 3
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string)

In [4]:
train_audio_transforms = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(n_mels=128, sample_rate = 22050, n_fft = 512, win_length=int(22050*0.02), hop_length=int(22050*0.01)),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
            torchaudio.transforms.TimeMasking(time_mask_param=35)
        )


valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, length_wav),  (utterance, length_trans) in data:
        waveform = waveform[:length_wav]
        utterance = utterance[:length_trans]
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths



BATCH_SIZE = 4

def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation

    d = np.zeros((len(r) + 1) * (len(h) + 1), dtype=np.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(r)][len(h)]

def GreedyDecoder(output, labels, label_lengths, blank_label=0, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [5]:
hook = sy.TorchHook(torch)

In [6]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [18]:
bob_data = pd.read_csv("Data/PartB_Telugu/Dev/samples_450_a.tsv", sep='\t', header=None)
alice_data = pd.read_csv("Data/PartB_Telugu/Dev/samples_450_b.tsv", sep='\t', header=None)

In [43]:
bob_train_dataset = sy.BaseDataset(bob_data[0].values.reshape(-1,1), [text_transform.text_to_int(u.lower()) for u in bob_data[1].values]).send(bob)
alice_train_dataset = sy.BaseDataset(alice_data[0].values.reshape(-1,1), [text_transform.text_to_int(u.lower()) for u in alice_data[1].values]).send(alice)

In [44]:
alice_train_dataset

[PointerDataset | me:21611257798 -> alice:16438615156]

In [45]:
federated_train_dataset = sy.FederatedDataset([bob_train_dataset, alice_train_dataset])

AttributeError: 'numpy.ndarray' object has no attribute 'owner'

In [ ]:
federated_avg()

In [5]:
train_dataset = CodeSwitchDataset(lang='Telugu', mode="train")
validation_split = 0.2
shuffle_dataset = True
random_seed = 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers = 6,
                          sampler = train_sampler,
                          collate_fn = lambda x: data_processing(x, 'train'))

test_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers=6,
                          sampler=valid_sampler,
                          collate_fn=lambda x: data_processing(x, 'valid'))



In [6]:
device = torch.device('cuda')
torch.cuda.empty_cache()

In [57]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

In [58]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = torch.nn.Linear(n_features, n_targets)

In [59]:
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

In [60]:
boston_data[worker][0], boston_target[worker][0]

((Wrapper)>[PointerTensor | me:85048122123 -> jon:1319887525]
 	Tags: #boston_housing .. #data _boston_dataset: #boston #housing 
 	Shape: torch.Size([86, 13])
 	Description: .. _boston_dataset:...,
 (Wrapper)>[PointerTensor | me:64259313581 -> jon:80090265347]
 	Tags: #boston_housing .. #target _boston_dataset: #boston #housing 
 	Shape: torch.Size([86])
 	Description: .. _boston_dataset:...)

In [61]:
n_features

13

In [7]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 64, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[

            ResidualCNN(64, 64, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*64, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [8]:
def train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    LR = 0
    train_loss = 0

    avg_acc = 0
    acc = []
    wers = []
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in pbar:
        #bi, wav, label = batch_idx, wav, label
        for g in optimizer.param_groups:
            LR=g['lr']
        wav, labels, input_lengths, label_lengths = _data

        # input_lengths, label_lengths = torch.IntTensor(input_lengths), torch.IntTensor(label_lengths)
        wav = wav.to(device)
        # wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        # output = model(wav, input_lengths)   #(batch, time, n_class) [4, 911, 3]
        output = model(wav)

        output = F.log_softmax(output, dim=2)


        output = output.transpose(0,1)
        
        # print(labels, label_lengths)
        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()
        scheduler.step()
        #print(loss)
        total_loss+=loss

        iter_meter.step()
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        print(decoded_preds, decoded_targets)
        print("preds: ", "".join(decoded_preds))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            wers.append(wer(decoded_targets[j], decoded_preds[j]))
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("accuracy/train_accuracy", avg_acc, epoch)
        writer.add_scalar('accuracy/train_loss', loss.item(), iter_meter.get())
        writer.add_scalar('CTCLoss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        writer.add_scalar("Learning Rate", LR, epoch)

        writer.add_scalar("WER", wer(decoded_targets[j], decoded_preds[j]), iter_meter.get())
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
    
    # for g in optimizer.param_groups:
    #   g['lr'] = g['lr']/LEARNING_ANNEAL
            
    #print(decoded_preds[0])
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join("fed_averaging_ckpt/", ckpt_model_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, ckpt_model_path)
        model.to(device).train()

In [9]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    test_loss=0
    acc = []
    with torch.no_grad():
        for batch_idx, _data in enumerate(test_loader):
            inputs, labels, input_lengths, label_lengths = _data 
            inputs, labels = inputs.to(device), labels.to(device)
            # output = model(inputs, input_lengths)  # (batch, time, n_class)
            output=model(inputs)
            output = F.log_softmax(output, dim=2)
            
            output = output.transpose(0, 1) # (time, batch, n_class)
            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)
            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
            for j in range(len(decoded_preds)):
                s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
                acc.append(s.ratio())

            avg_acc = sum(acc)/len(acc)
            writer.add_scalar("test_accuracy", avg_acc, epoch)
            writer.add_scalar('test_loss', test_loss, epoch)
            print("Test Accuracy: {}, Test loss: {}".format(avg_acc, test_loss))
        
    #print(decoded_targets)
    #print(decoded_preds)

def load_checkpoint(model, optimizer, filename='checkpoint.pth.tar'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 1
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

In [10]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

device = 'cuda'

hparams = {
        "n_cnn_layers": 4,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 4,
        "n_feats": 128,
        "stride": 2,
        "dropout": 0.1,
        "learning_rate": 5e-4,
        "batch_size": 4,
        "epochs": 60
    }

In [122]:
epoch_num = 1
model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)


model = model.to(device)
criterion = nn.CTCLoss(blank=0, reduction='mean').to(device)
epochs = 60

optimizer = optim.Adam(model.parameters(), hparams['learning_rate'])
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
    max_lr=hparams['learning_rate'],
    steps_per_epoch=int(len(train_loader)),
    epochs=hparams['epochs'],
    anneal_strategy='linear')

In [11]:

#model, optimizer, epoch_num = load_checkpoint(model, optimizer, "checkpoints/ckpt_epoch_30_batch_id_1645.pth")

print(epoch_num)
writer = SummaryWriter('fed_avg_logs/')
iter_meter = IterMeter()



# Commented out IPython magic to ensure Python compatibility.
# %load_ext tensorboard
# %tensorboard --logdir=train_logs

model.to(device)
model.train()

1


SpeechRecognitionModel(
  (cnn): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

In [33]:

for epoch in range(1, epochs+1):
    train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler)
    test(model, device, test_loader, criterion, epoch, writer)

model.eval().cpu()
save_model_filename = "final_epoch_client1" + str(epoch + 1)  + ".model"
save_model_path = os.path.join("fed_averaging_ckpt", save_model_filename)
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, save_model_path)

print("\nDone, trained model saved at", save_model_path)


  0%|          | 0/1645 [00:00<?, ?it/s]


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dexter/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/dexter/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "<ipython-input-25-1a85411fba24>", line 22, in <lambda>
    collate_fn = lambda x: data_processing(x, 'train'))
  File "<ipython-input-24-ae7434f7f734>", line 18, in data_processing
    for (waveform, _, length_wav),  (utterance, length_trans) in data:
ValueError: too many values to unpack (expected 3)


In [12]:
import syft as sy

In [13]:
hook = sy.TorchHook(torch)

In [14]:
bob = sy.VirtualWorker(hook=hook, id='bob')
alice = sy.VirtualWorker(hook=hook, id='alice')

In [15]:
hparams

{'n_cnn_layers': 4,
 'n_rnn_layers': 5,
 'rnn_dim': 512,
 'n_class': 4,
 'n_feats': 128,
 'stride': 2,
 'dropout': 0.1,
 'learning_rate': 0.0005,
 'batch_size': 4,
 'epochs': 60}

In [131]:
%%time
federated_dataset = CodeSwitchDataset(lang='Telugu', mode="train").federate((bob, alice)) ## IMPORANT: train + test

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 170226 and 166412 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:612

In [137]:
dat = CodeSwitchDataset(lang='Telugu', mode="train")

In [138]:
for i in dat:
    print(i)

((array([-0.00173453, -0.00094112,  0.00236711, ..., -0.00213112,
        0.00039463,  0.00406061], dtype=float32), 22050, 170226), ('SSTTEEETTTEETTTTTTTTTTTTSTTTTTTTSSSTTSSSSTTEEETTTEETTTTTTTTTTTTSTTTTTTTSSSTTSSSSTTEEETTTEETTTTTTTTTTTTSTTTTTTTSSS', 39, 113))
((array([-0.00037996, -0.00033962, -0.00027104, ...,  0.04222445,
        0.03440117,  0.02218605], dtype=float32), 22050, 166412), ('SSTTTTTEEETTTTTTTTTTTTTTSSSSSSTTTTTEEETTTTTTTTTTTTTTSSSSSSTTTTTEEETTTTTTTTTTTTTTS', 28, 81))
((array([0.0050178 , 0.00448516, 0.00182789, ..., 0.01210684, 0.01618266,
       0.01659433], dtype=float32), 22050, 84231), ('SSEEEEETTTTTTTETTTSSSSEEEEETTTTTTTETTTSSSSEEEEETTTTTTTETTTSSSSEEEEETTTTTTTETTTSSSSEEEEETTTTTTTETTTSSSSEEEEETTTTTTTET', 20, 116))
((array([ 0.00085677,  0.00072619, -0.00011591, ..., -0.00263321,
       -0.00097778,  0.00130088], dtype=float32), 22050, 225572), ('SSSSSEEETTTSSSSEEETTTTTTSTTTTTTTTTTTTTSTTTTTTTTTTTSSSSSSSEEETTTSSSSEEETTTTTTSTTTTTTTTTTTTTSTTTTTTSSSSSEEETTTSSSSEEETTTTTTST

KeyboardInterrupt: 

In [16]:
class CodeSwitchDataset(Dataset):
    def __init__(self, lang, mode = "train", shuffle=True):
        self.mode = mode
        # data path
        self.lang = lang
        if self.lang == "Gujarati":
            self.max_len = 0
        elif self.lang == "Telugu":
            self.max_len = 617621
        elif self.lang == 'Tamil':
            self.max_len = 0
        else:
            raise Exception("Check Language")
        if self.mode == "train":
            self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        elif self.mode == "test":
            self.path = self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        else:
            raise Exception("Incorrect mode")
        self.file_list = os.listdir(os.path.join(self.path, 'Audio'))
        self.shuffle=shuffle
        self.csv_file = pd.read_csv(self.path + 'Transcription_LT_Sequence_Frame_Level_200_actual.tsv', header=None, sep='\t')
        self.input_length = []
        self.label_length = []
        

    def __len__(self):
        return len(self.csv_file)

    def pad(self, wav, trans, max_len):
        orig_len = len(wav)
        while len(wav) < max_len:
            diff = max_len - len(wav)
            ext = wav[:diff]
            wav = np.append(wav, wav[:diff])
            ratio = int(len(trans)*diff/len(wav))
            trans +=trans[:ratio]
        return wav, trans

    def preprocess(self, wav, sr, trans):

        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = [np.log(1 + x) for x in out]
        return np.array(out), trans

    def __getitem__(self, idx):
        file_name = self.csv_file[0][idx]
        trans = self.csv_file[1][idx]
        
        #wav, sr = librosa.load(glob(self.path + 'Audio/*'+ str(file_name) + '.wav')[0])
        #length_wav, length_trans = len(wav), len(trans)
        #wav, trans = self.pad(wav, trans, self.max_len)
        ##if len(set(trans)) > 2:
        #    label = 1
        #elif len(set(trans)) == 1 or len(set(trans)) == 2:
        #    label = 0
        #else:
        #    raise Exception("Check transcript")
            
        if self.mode =="train":
            return file_name, file_name
        elif self.mode == "test":
            return file_name
        else:
            raise Exception("Incorrect Mode")

In [96]:
def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    """
    for (waveform, _, length_wav),  (utterance, length_trans) in data:
        waveform = waveform[:length_wav]
        utterance = utterance[:length_trans]
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths
    """
    files = []
    trans = []
    for file_name, transcript in data:
        print("fffffff", file_name)
        print("ttttttt", transcript)
        files.append(file_name)
        trans.append(transcript)
    
    return files, trans

In [126]:
federated_train_loader = sy.FederatedDataLoader(federated_dataset,
                                                batch_size=BATCH_SIZE,
                                                sampler=train_sampler,
                                                num_workers = 6,
                                                collate_fn=lambda x: data_processing(x, 'train'))

federated_test_loader = sy.FederatedDataLoader(federated_dataset,
                                               batch_size=BATCH_SIZE,
                                               sampler = valid_sampler,
                                               num_workers = 6,
                                               collate_fn=lambda x: data_processing(x, 'valid'))

In [127]:
federated_dataset

FederatedDataset
    Distributed accross: bob, alice
    Number of datapoints: 1047

In [128]:
for idx in federated_train_loader:
    print('f', idx)

TypeError: iteration over a 0-d tensor

In [113]:
idx[0]

[(Wrapper)>[PointerTensor | me:93614304625 -> alice:35463363193],
 (Wrapper)>[PointerTensor | me:31853028333 -> alice:57807864418],
 (Wrapper)>[PointerTensor | me:38481110105 -> alice:61205180234]]

In [103]:
idx[0][1].get()

tensor(102280401)

In [104]:
idx[0][1].send(alice)

AttributeError: 'Tensor' object has no attribute 'child'

In [51]:
idx[1][0].get()

tensor(102280394)

In [ ]:
## Testing

In [11]:
for d1 in enumerate(dat):
    print(d1)

(0, (10162, tensor([1., 1., 3., 3., 2., 2., 2., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3.,
        3., 1., 1.])))
(1, (10210, tensor([1., 1., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 1., 1., 1.])))
(2, (10234, tensor([1., 1., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3.,
        1., 1.])))
(3, (10288, tensor([1., 1., 1., 1., 1., 2., 2., 2., 3., 3., 3., 1., 1., 1., 1., 2., 2., 2.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(4, (10443, tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 2., 2., 2., 3., 3., 1.,
        2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1.])))
(5, (10450, tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1., 1., 1., 1., 1

(129, (380257, tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 1., 3., 3., 2., 3., 3., 3., 2., 2., 3., 3., 3., 3., 1.])))
(130, (380333, tensor([1., 1., 3., 3., 3., 3., 2., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1.])))
(131, (380339, tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 1., 1.])))
(132, (380398, tensor([1., 1., 2., 2., 2., 3., 3., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 2.,
        2., 2., 3., 2., 2., 2., 2., 3., 1., 1.])))
(133, (380447, tensor([1., 1., 3., 3., 3., 3., 2., 2., 3., 3., 1., 2., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(134, (390094, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 2.,
        2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(135, (390467, tensor([1., 1., 1., 3., 3., 3

(261, (780148, tensor([1., 1., 3., 3., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 1., 1., 1., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
        2., 2., 2., 2., 1., 1., 3., 3., 3., 1., 1.])))
(262, (780158, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3.,
        3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 3.,
        3., 3., 3., 1., 1.])))
(263, (780209, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 3.,
        3., 3., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 3., 1., 2., 2., 2., 3.,
        3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3.,
        1., 1., 3., 3., 3., 3., 1., 1.])))
(264, (840024, tensor([1.,

(378, (1870322, tensor([1., 1., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.,
        1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 2., 2.,
        2., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(379, (1880020, tensor([1., 1., 1., 2., 2., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(380, (1880068, tensor([1., 1., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 1., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(381, (1880069, tensor([1., 1., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 3., 3., 2., 2., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(382, (1880202, tensor([1., 1., 2., 2., 2., 2., 2., 2., 3., 3., 3., 1., 3., 3., 3., 2., 2., 3.,
        3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(383, (188044

(508, (2240423, tensor([1., 1., 2., 2., 3., 3., 3., 1., 3., 3., 3., 3., 1., 1., 1., 1., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(509, (2240431, tensor([1., 1., 3., 3., 3., 3., 3., 1., 1., 2., 2., 2., 2., 1., 3., 3., 3., 2.,
        2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 1., 1., 1.])))
(510, (2250055, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(511, (2250157, tensor([1., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3.])))
(512, (2250229, tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(513, (2250321, tensor([1., 1., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 2., 3., 3., 1., 1.])))
(514, (2250345, tensor([1., 1., 2., 2., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        1., 1.])))
(515, (2250400, tensor([1., 1., 3., 3., 3., 3.,

(643, (100380453, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 1., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 1.])))
(644, (100390076, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 3., 3., 3., 1., 1., 1.])))
(645, (100390133, tensor([1., 1., 1., 1., 3., 3., 3., 1., 1., 3., 3., 3., 2., 3., 3., 3., 3., 1.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 1.])))
(646, (100390164, tensor([1., 1., 1., 2., 3., 3., 1., 1., 3., 1., 1., 1., 1., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 1., 1., 1., 1., 3., 3., 3.,
        3., 1., 1.])))
(647, (100390429, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 1., 2., 2., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(648, (100400017, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2.,
        3., 3., 3., 3., 3., 3., 3., 3

(769, (100770318, tensor([1., 1., 1., 3., 3., 3., 3., 3., 2., 2., 1., 1., 1., 3., 3., 3., 3., 3.,
        1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(770, (100770428, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3.,
        3., 1., 3., 3., 3., 2., 2., 3., 3., 1., 3., 3., 1., 1.])))
(771, (100780229, tensor([1., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 1., 1., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])))
(772, (100780276, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1., 1., 3., 3., 3., 3., 3., 3., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1.,
        1., 1., 1., 3., 3., 3., 2., 2., 2., 2., 3., 3., 3., 3., 3., 1., 1.])))
(773, (100780289, tensor([1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3., 1., 3., 3.,
      

(884, (101880263, tensor([1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 3., 2., 2., 1., 1., 1., 3., 3.,
        3., 3., 3., 3., 1., 1., 1., 3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 1.])))
(885, (101880396, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 1., 3., 3.,
        3., 3., 3., 3., 2., 2., 1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3.,
        1., 1.])))
(886, (101890004, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 1.])))
(887, (101890104, tensor([1., 1., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(888, (101890108, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(889, (101890177, tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3

(1002, (102220391, tensor([1., 1., 3., 3., 3., 3., 2., 2., 2., 1., 1., 1., 2., 2., 3., 3., 3., 2.,
        2., 3., 3., 3., 3., 3., 3., 3., 3., 1.])))
(1003, (102220392, tensor([1., 1., 3., 3., 3., 2., 2., 2., 2., 3., 3., 3., 3., 1., 1., 1., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 1.])))
(1004, (102220455, tensor([1., 1., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1.])))
(1005, (102230072, tensor([1., 1., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 1.])))
(1006, (102230085, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3.,
        3., 1.])))
(1007, (102230159, tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 2., 2., 2., 3., 3.,
        3., 1., 1.])))
(1008, (102230198, tensor([1., 1., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 3., 3., 1., 1.])))
(1009, (102230229, tensor([1., 1., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3.,

KeyError: 1047

In [12]:
d1[1]

(102280436,
 tensor([1., 1., 3., 3., 3., 3., 3., 1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
         3., 1., 1.]))

In [81]:
len(d1[1])

2

In [15]:
dat = CodeSwitchDataset('Telugu')

In [13]:
train_loader = DataLoader(dat,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers = 6,
                          sampler = train_sampler,
                          collate_fn = lambda x: data_processing(x, 'train'))

In [17]:
for batch_idx, (files, trans) in enumerate(dat):
    print(batch_idx, files, trans)

0 10162 tensor([1., 1., 3., 3., 2., 2., 2., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3.,
        3., 1., 1.])
1 10210 tensor([1., 1., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 1., 1., 1.])
2 10234 tensor([1., 1., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3.,
        1., 1.])
3 10288 tensor([1., 1., 1., 1., 1., 2., 2., 2., 3., 3., 3., 1., 1., 1., 1., 2., 2., 2.,
        3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])
4 10443 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 2., 2., 2., 3., 3., 1.,
        2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1.])
5 10450 tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 1.,

110 340223 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2.,
        3., 3., 3., 3., 3., 1., 1.])
111 340295 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 1.,
        1.])
112 350141 tensor([1., 1., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3., 2., 2., 3., 3., 2.,
        2., 3., 3., 3., 1., 1.])
113 350150 tensor([1., 1., 1., 1., 3., 2., 2., 2., 2., 2., 2., 2., 1., 1., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 3., 3.,
        3., 3., 1.])
114 360002 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 3.,
        3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3.,
        1., 1.])
115 360052 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1., 2., 2.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 1., 1., 3., 3., 3., 3.,
        3., 3., 3., 1., 1., 2., 2., 2., 3., 3., 3., 1., 1.])
116 360156 tensor([1., 1., 1., 1

231 700260 tensor([1., 1., 3., 3., 2., 2., 3., 3., 3., 3., 3., 1., 1., 1., 1., 3., 3., 3.,
        3., 3., 3., 3., 3., 1.])
232 700322 tensor([1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1., 2.,
        2., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 1., 1.])
233 700331 tensor([1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        1.])
234 700358 tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 1.])
235 700462 tensor([1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 1., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        1., 1.])
236 710068 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2.,
        2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.,
        1., 1., 3., 3., 3., 1., 1.])
237 710195 tensor([1., 1., 3., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
 

345 1350181 tensor([1., 1., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 1., 1.])
346 1360115 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 1.])
347 1360276 tensor([1., 1., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 1.])
348 1360308 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])
349 1370018 tensor([1., 1., 1., 3., 2., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 1., 1., 1.])
350 1370257 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3.,

455 2090377 tensor([1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2.,
        3., 3., 3., 3., 3., 2., 2., 1., 1.])
456 2090441 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.])
457 2100310 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 2., 3., 3., 3., 3., 1.])
458 2100455 tensor([1., 1., 1., 1., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 3.,
        3., 3., 3., 3., 3., 2., 2., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 1.])
459 2110055 tensor([1., 1., 3., 3., 3., 3., 3., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 1., 2., 2., 2.,
        3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 1.])
460 2120193 tensor([1., 1., 1., 1., 1., 1., 1., 3., 3.,

...

688 100480295 tensor([1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 2., 2., 1., 1., 3., 3., 3.,
        3., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1.])
689 100490079 tensor([1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 1.,
        1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 2., 3., 3.,
        3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 1., 1.])
690 100490111 tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 3., 3., 3.,
        3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 3., 3., 3., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 1.])
691 100490203 tensor([1., 1., 1., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 2., 2., 2., 2., 1.,
        1., 1., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
        3., 3., 2., 2., 2., 1., 1.])
692 100490256 tensor([1., 1., 1., 1., 1., 3., 3., 3., 2., 2., 2., 1., 2., 3., 3., 1., 1.

808 100950334 tensor([1., 1., 1., 3., 3., 3., 1., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 2., 2., 2., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])
809 100950360 tensor([1., 1., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        3., 3., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.])
810 100950429 tensor([1., 1., 2., 2., 2., 2., 2., 3., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1.,
        3., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 2., 2., 3., 3., 3., 3., 3., 1., 1., 1.])
811 100960187 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3.,
        3., 1., 3., 3., 3., 3., 1., 1.])
812 100960276 tensor([1., 1., 1., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3.

921 101980148 tensor([1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 1., 2., 2., 2.,
        2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 3.,
        3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 1., 1.])
922 101980364 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3.,
        3., 3., 3., 3., 3., 1.])
923 101990072 tensor([1., 1., 1., 3., 3., 3., 3., 2., 2., 3., 2., 2., 3., 3., 2., 3., 2., 3.,
        3., 3., 3., 3., 1., 1.])
924 102000366 tensor([1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 1., 1.])
925 102010458 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 2., 2., 2., 3.,
        3., 3., 3., 3., 3., 1., 1.])
926 102020302 tensor([1., 2., 2., 2., 3., 3.,

1046 102280436 tensor([1., 1., 3., 3., 3., 3., 3., 1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 1., 1.])


KeyError: 1047

In [18]:
files

102280436

In [19]:
trans

tensor([1., 1., 3., 3., 3., 3., 3., 1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 1., 1.])

In [14]:
for batch_idx, (files, trans) in enumerate(train_loader):
    print(batch_idx, files, trans)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dexter/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/dexter/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "<ipython-input-13-73a1c041dae2>", line 6, in <lambda>
    collate_fn = lambda x: data_processing(x, 'train'))
  File "<ipython-input-3-ae7434f7f734>", line 18, in data_processing
    for (waveform, _, length_wav),  (utterance, length_trans) in data:
TypeError: 'numpy.int64' object is not iterable


In [71]:
batch_idx

208

In [72]:
files

[1020200, 102220391, 130151, 100580219]

In [73]:
trans

['SSSTTTTTTTTTTTTTTSTTTTTTTTSTTTTTTTTTTTSTTTTTTTTTTTTTTTEETTTTS',
 'SSTTTTEEESSSEETTTEETTTTTTTTS',
 'SSTTTEETTTTEETTTTTTTTS',
 'SSSTTEEEEETTTTSTTTTTTTTTTTSS']

In [20]:
d = CodeSwitchDataset('Telugu')

In [27]:
federated_dataset = CodeSwitchDataset(lang='Telugu', mode="train").federate((bob, alice)) ## IMPORANT: train + test

Remove this print statement tensor([  10162,   10210,   10234,   10288,   10443,   10450,   20052,   20249,
          20298,   30204,   30335,   40283,   40286,   40435,   50070,   50426,
          50456,   60373,   60386,   60387,   70012,   70058,   70086,   70172,
          70198,   70205,   70324,   90100,   90118,   90182,   90296,  100275,
         100340,  100350,  110012,  110025,  110115,  110139,  110246,  110285,
         110405,  110481,  120487,  130043,  130089,  130146,  130151,  130193,
         160009,  160036,  160370,  160437,  170048,  180095,  180308,  180310,
         180317,  190053,  190201,  190338,  190348,  190417,  190422,  190443,
         220111,  220195,  220238,  230008,  230094,  230371,  240194,  240210,
         240280,  240333,  240386,  240433,  250033,  250105,  250115,  250191,
         250294,  250341,  250355,  280176,  280299,  280347,  280366,  280380,
         300074,  300089,  300159,  300271,  300294,  310142,  310259,  320021,
         320

In [29]:
for i, dat in enumerate(federated_dataset):
    print(i)

KeyError: 0